In [7]:
import georinex as gr
import numpy as np
import json

In [8]:
# Указываем путь к файлу SP3
sp3_file_path = '/Users/karimvafin/Desktop/GSR_1-3D_24031406.sp3'

In [9]:
# Загружаем данные из файла SP3
obs = gr.load(sp3_file_path, use='G').dropna(dim='time', how='all')
obs

<xarray.Dataset>
Dimensions:   (time: 288, sv: 119, ECEF: 3)
Coordinates:
  * time      (time) datetime64[ns] 2024-03-14T06:00:00 ... 2024-03-17T05:45:00
  * sv        (sv) <U3 'C06' 'C07' 'C08' 'C09' 'C10' ... 'G32' 'J02' 'J03' 'J04'
  * ECEF      (ECEF) <U1 'x' 'y' 'z'
Data variables:
    position  (time, sv, ECEF) float64 -6.378e+03 4.012e+04 ... -1.816e+04
    clock     (time, sv) float64 271.9 -917.3 63.04 ... -2.998 -0.188 50.67
    velocity  (time, sv, ECEF) float64 0.0 7.145e-67 ... 3.308e-33 4.913e-62
    dclock    (time, sv) float64 1.225e-321 6.943e-310 ... 3.331e+35 1.246e-47
Attributes:
    t0:          2024-03-14 06:00:00
    Nepoch:      288
    coord_sys:   IGS20
    orbit_type:  EXT
    agency:       GSR

In [10]:
times_obs = obs.time.data

In [11]:
dct_to_save = {}
for t in times_obs:
    obs_pd_temp = obs.sel(time=t).to_dataframe().reset_index().dropna()
    obs_pd_temp = obs_pd_temp.groupby('sv').agg(list).reset_index()
    del obs_pd_temp['time']
    del obs_pd_temp['ECEF']
    obs_pd_temp['clock'] = obs_pd_temp['clock'].apply(lambda x: x[0])
    obs_pd_temp['dclock'] = obs_pd_temp['dclock'].apply(lambda x: x[0])
    obs_pd_temp = {str(t): obs_pd_temp.to_dict('list')}
    dct_to_save.update(obs_pd_temp)

In [12]:
with open('/Users/karimvafin/Desktop/satellite_data_1.json', 'w') as f:
    json.dump(dct_to_save, f)
